## class State

This class encodes the positions of the balls, dimensions of the table, and the state of the game.

In [2]:
import numpy as np
import dataclasses


@dataclasses.dataclass
class State():
    pass